In [1]:
# ### Tracing (optional)
# secret keys 
import os




In [2]:
### Build Index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

### from langchain_cohere import CohereEmbeddings

# Set embeddings
embd = OpenAIEmbeddings()

# Docs to index
urls = [
    # "https://lilianweng.github.io/posts/2023-06-23-agent/",
    # "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    # "https://ag.ny.gov/coronavirus/coronavirus-tenants-rights#nys-eviction",
    # "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    # "https://abovethelaw.com/2024/07/trump-immunity-opinion-textualist-originalist/",
    # "https://abovethelaw.com/2024/07/nixons-former-lawyer-is-getting-a-kick-out-of-the-supreme-courts-presidential-immunity-ruling/",
    # "https://abovethelaw.com/2024/07/ysl-rico-case-put-on-hold-to-search-for-new-judge/",
    "https://ag.ny.gov/publications/residential-tenants-rights-guide", 
    "https://www.curbed.com/article/new-york-city-landlord-tenant-law-rights.html",
    "https://www.legalservicesnyc.org/what-we-do/practice-areas-and-projects/housing",
    "https://hcr.ny.gov/fire-damaged-vacate-order-apartments",
    "https://www.consumerfinance.gov/ask-cfpb/what-should-i-do-if-my-house-is-destroyed-in-a-natural-disaster-en-1521/"
    
]

# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

#persist_directory = "vectorstore_storage"

# Add to vectorstore
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embd,
    #persist_directory = persist_directory
)
# vectorstore.persist()

# vectorstore = Chroma(
#     collection_name="rag-chroma",
#     #embedding=embd,
#     persist_directory=persist_directory
# )


retriever = vectorstore.as_retriever()

In [3]:
#utulize saved vector store 
# from langchain_community.vectorstores import Chroma
# from langchain_openai import OpenAIEmbeddings

# # Set embeddings
# embd = OpenAIEmbeddings()

# # Define persist directory
# persist_directory = "/home/sunny/Documents/Law-Ai-Advisor-/vectorstore_storage"

# # Load the persisted vector store
# vectorstore = Chroma(
#     collection_name="rag-chroma",
#     persist_directory=persist_directory,
#     embedding_function=embd.embed_documents
# )

# # Set the embedding function
# #vectorstore.embedding_function = embd.embed_documents

# # Create the retriever
# retriever = vectorstore.as_retriever()

# # Example query
# query = "Explain the concept of prompt engineering"
# results = retriever.get_relevant_documents(query)


# # Print results
# for result in results:
#     print(result.metadata['source'], result.text)

In [4]:
#Keeping rag simple for now till chain is done. 

# import os
# import json
# from bs4 import BeautifulSoup
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma

# class Document:
#     def __init__(self, text, metadata=None):
#         self.page_content = text
#         self.metadata = metadata if metadata is not None else {}

# def parse_html(html_content):
#     soup = BeautifulSoup(html_content, 'html.parser')
#     return soup.get_text()

# def load_and_process_json_files(directory):
#     all_documents = []
#     for filename in os.listdir(directory):
#         if filename.endswith(".json"):
#             file_path = os.path.join(directory, filename)
#             with open(file_path, 'r', encoding='utf-8') as file:
#                 try:
#                     data = json.load(file)
#                     opinions = data.get('casebody', {}).get('opinions', [])
#                     text = opinions[0].get('text', '') if opinions else ''
#                     metadata = {
#                         'source': 'JSON',
#                         'filename': filename,
#                         'case_id': data.get('id', ''),
#                         'case_name': data.get('name', ''),
#                         'decision_date': data.get('decision_date', ''),
#                         'court': data.get('court', {}).get('name', ''),
#                         'jurisdiction': data.get('jurisdiction', {}).get('name', '')
#                     }
#                     document = Document(text, metadata)
#                     all_documents.append(document)
#                 except json.JSONDecodeError:
#                     print(f"Error decoding JSON from file: {filename}")
#     return all_documents

# def read_and_parse_html_files(directory):
#     all_documents = []
#     for filename in os.listdir(directory):
#         if filename.endswith(".html"):
#             file_path = os.path.join(directory, filename)
#             with open(file_path, 'r', encoding='utf-8') as file:
#                 html_content = file.read()
#                 text = parse_html(html_content)
#                 metadata = {
#                     'source': 'HTML',
#                     'filename': filename
#                 }
#                 document = Document(text, metadata)
#                 all_documents.append(document)
#     return all_documents

# def split_documents(documents):
#     split_docs = []
#     for doc in documents:
#         splits = text_splitter.split_text(doc.page_content)
#         for split in splits:
#             split_docs.append(Document(split, doc.metadata))
#     return split_docs

# # Set embeddings
# embd = OpenAIEmbeddings()
# # Split text using RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

# # Define the directories containing JSON and HTML files
# json_directory_path = '/home/sunny/Documents/Adaptive_rag/14/json'  # Update this path as needed
# html_directory_path = '/home/sunny/Documents/Adaptive_rag/14/html'  # Update this path as needed

# # Read and parse JSON files
# json_documents = load_and_process_json_files(json_directory_path)

# # Read and parse HTML files
# html_documents = read_and_parse_html_files(html_directory_path)

# # Combine JSON and HTML documents
# all_documents = json_documents + html_documents

# # Split documents into smaller chunks
# doc_splits = split_documents(all_documents)

# # Create the vector store from the documents
# vectorstore = Chroma.from_documents(
#     documents=doc_splits,
#     collection_name="rag-chroma",
#     embedding=embd
# )

# # Set up the retriever
# retriever = vectorstore.as_retriever()

# # Optional: Print or process the first few entries to verify
# for doc in doc_splits[:5]:
#     print(doc.page_content[:200])  # Print the first 200 characters of each document split


In [5]:
### Router

from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["Complex", "Moderate", "Simple"] = Field(
        ...,
        #description="Given a user question choose to route it to web search or a vectorstore.",
        description="Given a user question choose to route it to Complex if the query is highly complex, Moderate if the query is moderately complex, and use Simple if it is simple or irrelavant to anything else in the vectorestore.", 
    )

# LLM with function call
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
#llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
llm = ChatOpenAI(model="gpt-4o", temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt
system = """You are an expert at routing a user question. There are three tiers Complex, Moderate, and Simple.
The Complex are prompts with alot of nuance and are diffictult to solve. Moderate are easier but with some complexity. Simple can easily be solved or is irrelvant to anything in the vectore store.
complex_indicators = ['explain', 'detailed', 'comprehensive', 'complex', 'in-depth', 'analysis'],
moderate_indicators = ['how', 'why', 'steps', 'process', 'method', 'moderate', 'overview', 'general'],
simple_indicators = ['what', 'when', 'who', 'define', 'simple', 'basic', 'example']

Example of a simple, irrelevant prompt: 
- Prompt: "What is a MacBook?"
- Answer: "A MacBook is a line of laptop computers developed and manufactured by Apple Inc."

"""

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
print(
    question_router.invoke(
        {"question": "Explain to me about Trump's legal issues?"}
    )
)
print(question_router.invoke({"question": "What did the supreme court recently say?"}))
print(question_router.invoke({"question": "steps in the proccess in prompting?"}))
print(question_router.invoke({"question": "Lebron James?"}))

datasource='Complex'
datasource='Simple'
datasource='Moderate'
datasource='Simple'


In [6]:
### Retrieval Grader


# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM with function call
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
#llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
llm = ChatOpenAI(model="gpt-4o", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "agent memory"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

/home/sunny/anaconda3/envs/llm/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


binary_score='no'


In [7]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
#llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
#llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
llm = ChatOpenAI(model="gpt-4o", temperature=0)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

I don't know. The provided context does not contain information about "agent memory."


In [8]:
# Define a simple prompt template
prompt_template = ChatPromptTemplate.from_messages(
    ["Question: {question} \nAnswer: "]
)

# Create the language model chain
llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm_chain = prompt_template | llm | StrOutputParser()

# Define the question
question = "Hi, how are you?"

# Run the model chain
generation = llm_chain.invoke({"question": question})
print(generation)


Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [9]:
### Hallucination Grader


# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# LLM with function call
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
#llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
llm = ChatOpenAI(model="gpt-4o", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='no')

In [10]:
### Answer Grader


# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )


# LLM with function call
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
#llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
llm = ChatOpenAI(model="gpt-4o", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# Prompt
system = """You are a grader assessing whether an answer addresses / resolves a question \n 
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='no')

In [11]:


### Question Re-writer

# LLM
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
#llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})



'What is your current state of well-being?'

In [12]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

In [13]:
from typing import List, Optional 

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: Optional list of documents
    """

    question: str
    generation: str
    documents: Optional[List[str]]

In [14]:
from langchain.schema import Document


def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}

def self_reflection_retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

def mod_generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

# testing no rag gen 
def rag_less_generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state
        llm (callable): The language model function to generate responses

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]

    # LLM generation
    generation = llm_chain.invoke({"question": question})
    return {"question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "question": question}


### Edges ###


def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == "Moderate":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "Moderate"
    elif source.datasource == "Complex":
        print("---ROUTE QUESTION TO RAG---")
        return "Complex"
    elif source.datasource == "Simple":
        print("---ROUTE QUESTION TO LLM---")
        return "Simple"


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [15]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
#workflow.add_node("web_search", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("self_reflection_retrieve", self_reflection_retrieve)  # self reflection retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generate
workflow.add_node("mod_generate", mod_generate)  # moderate generate
workflow.add_node("rag-less generate", rag_less_generate) 
workflow.add_node("transform_query", transform_query)  # transform_query

# Build graph
workflow.set_conditional_entry_point(
    route_question,
    {
        #"web_search": "web_search",
        #"vectorstore": "retrieve",
        #"llm": "rag-less generate",
        "Complex": "self_reflection_retrieve",
        "Moderate": "retrieve",
        "Simple": "rag-less generate",

    },
)
#workflow.add_edge("web_search", "generate")
#workflow.add_edge("Moderate", "retrieve")
workflow.add_edge("retrieve", "mod_generate")

workflow.add_edge("self_reflection_retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "self_reflection_retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()

In [16]:
from pprint import pprint

# Run
inputs = {
    "question": "What is Basketball?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO LLM---
---GENERATE---
"Node 'rag-less generate':"
'\n---\n'
('Basketball is a team sport in which two teams, typically consisting of five '
 'players each, compete to score points by shooting a ball through the '
 "opponent's hoop, which is a circular net suspended from a backboard. The "
 'game is played on a rectangular court with a hoop at each end. Points are '
 'scored by making field goals (worth two or three points, depending on the '
 'distance from the hoop) or free throws (worth one point each). The team with '
 'the most points at the end of the game wins. Basketball is governed by a set '
 'of rules that dictate how the game is played, including dribbling, passing, '
 'and shooting. It is a fast-paced and physically demanding sport that '
 'requires a combination of athleticism, strategy, and teamwork. The sport is '
 'popular worldwide and is played at various levels, from amateur to '
 'professional, including leagues such as the 

In [17]:
from pprint import pprint

# Run
inputs = {
    "question": "Tell me about YSL RICO put on hold to search for new judge?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO WEB SEARCH---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---GENERATE---
"Node 'mod_generate':"
'\n---\n'
"I don't know."


In [18]:
from pprint import pprint

# Run
inputs = {
    "question": "Tell me about prompting and the ways I can approach it?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO WEB SEARCH---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---GENERATE---
"Node 'mod_generate':"
'\n---\n'
"I don't know."


In [19]:
from pprint import pprint

# Run
inputs = {
    "question": "Who is actor of John Snow?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Available keys in the current output: {value.keys()}")
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        #pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO LLM---
---GENERATE---
"Available keys in the current output: dict_keys(['question', 'generation'])"
"Node 'rag-less generate':"
'\n---\n'
('The actor who portrays Jon Snow in the television series "Game of Thrones" '
 'is Kit Harington.')


In [20]:
inputs = {
    "question": "Elaborate about the complexity of prompting and the ways I can approach it?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'self_reflection_retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---
"Node 'grade_documents':"
'\n---\n'
---TRANSFORM QUERY---
"Node 'transform_query':"
'\n---\n'
---RETRIEVE---
"Node 'self_reflection_retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---
"Node 'grade_documents':"
'\n---\n'
---TRANSFORM QUERY---
"Node 'transform_query':"
'\n---\n'
---RETRIEVE---
"Node 'self_reflection_retrieve':"
'\n---\n'
---CH

KeyboardInterrupt: 

In [ ]:
inputs = {
    "question": "Tell me about Trump?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO LLM---
---GENERATE---
"Node 'rag-less generate':"
'\n---\n'
('Donald J. Trump is an American businessman, television personality, and '
 'politician who served as the 45th President of the United States from '
 'January 20, 2017, to January 20, 2021. Born on June 14, 1946, in Queens, New '
 'York City, Trump is the fourth of five children of Fred and Mary Trump. He '
 'attended the Wharton School of the University of Pennsylvania, where he '
 'graduated with a degree in economics.\n'
 '\n'
 'Before his presidency, Trump was known for his real estate ventures, '
 'particularly in New York City, where he developed, owned, and managed '
 'numerous properties, including the famous Trump Tower. He also gained fame '
 'as the host of the reality TV show "The Apprentice," which aired from 2004 '
 'to 2015.\n'
 '\n'
 "Trump's 2016 presidential campaign was marked by its unconventional style, "
 'populist rhetoric, and the slogan "Make America Great Aga

In [21]:
inputs = {
    "question": "Where is the courthouse in Queens?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO LLM---
---GENERATE---
"Node 'rag-less generate':"
'\n---\n'
('The main courthouse in Queens is the Queens County Criminal Court, located '
 'at 125-01 Queens Blvd, Kew Gardens, NY 11415. This courthouse handles '
 'criminal cases. For civil matters, the Queens County Civil Court is located '
 'at 89-17 Sutphin Blvd, Jamaica, NY 11435. Additionally, the Queens County '
 'Supreme Court, which handles both civil and criminal cases, is also located '
 'at 88-11 Sutphin Blvd, Jamaica, NY 11435.')


In [22]:
inputs = {
    "question": "How do I get a lease for a public housing apartment in NY?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO WEB SEARCH---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---GENERATE---
"Node 'mod_generate':"
'\n---\n'
('To get a lease for a public housing apartment in New York, you need to apply '
 'through the local housing authority, which manages public housing '
 'developments. The process involves meeting eligibility requirements, '
 'including income limits and family size. For more detailed information, you '
 'can refer to the guides provided by the New York City Department of Housing '
 'Preservation and Development (HPD) and the state’s Division of Homes and '
 'Community Renewal (DHCR).')


In [23]:
inputs = {
    "question": "Explain how do I file a claim for damages if my apartment was destroyed in a fire in NY?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'self_reflection_retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION DOES NOT ADDRESS QUESTION---
"Node 'generate':"
'\n---\n'
---TRANSFORM QUERY---
"Node 'transform_query':"
'\n---\n'
---RETRIEVE---
"Node 'self_reflection_retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
---CHEC

GraphRecursionError: Recursion limit of 25 reachedwithout hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.

In [ ]:
inputs = {
    "question": "Explain how do I file a claim for damages if my apartment was destroyed in a fire in NY?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'self_reflection_retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION DOES NOT ADDRESS QUESTION---
"Node 'generate':"
'\n---\n'
---TRANSFORM QUERY---
"Node 'transform_query':"
'\n---\n'
---RETRIEVE---
"Node 'self_reflection_retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
---CHEC

GraphRecursionError: Recursion limit of 25 reachedwithout hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.

In [24]:
inputs = {
    "question": "Explain how to claim for fire damages in NY?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'self_reflection_retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---
"Node 'grade_documents':"
'\n---\n'
---TRANSFORM QUERY---
"Node 'transform_query':"
'\n---\n'
---RETRIEVE---
"Node 'self_reflection_retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---
"Node 'grade_documents':"
'\n---\n'
---TRANSFORM QUERY---
"Node 'transform_query':"
'\n---\n'
---RETRIEVE---
"Node 'self_reflection_retrieve':"
'\n---\n'
---CH

GraphRecursionError: Recursion limit of 25 reachedwithout hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.